Update Code

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load datasets
def load_and_prepare_data():
    # Load timekeeping data
    timekeeping_df = pd.read_csv('sample_timekeeping.csv')
    
    # Load payroll data
    payroll_df = pd.read_csv('sample_payroll.csv')
    
    # Ensure proper data types
    payroll_df['employee_id'] = payroll_df['employee_id'].astype(int)
    payroll_df['year'] = payroll_df['year'].astype(int)
    payroll_df['pay_rate'] = payroll_df['pay_rate'].astype(float)
    
    # Convert date column to datetime and extract year
    timekeeping_df['date'] = pd.to_datetime(timekeeping_df['date'])
    timekeeping_df['year'] = timekeeping_df['date'].dt.year
    
    return timekeeping_df, payroll_df

# Process time punches to calculate work duration
def process_time_punches(df):
    # Convert punch times to datetime
    time_columns = ['punchin1', 'punchout1', 'punchin2', 'punchout2', 'punchin3', 'punchout3']
    
    for col in time_columns:
        # Handle missing values
        df[col] = pd.to_datetime(df[col], errors='coerce')
    
    # Calculate duration for each punch pair (in hours)
    df['duration1'] = (df['punchout1'] - df['punchin1']).dt.total_seconds() / 3600
    df['duration2'] = (df['punchout2'] - df['punchin2']).dt.total_seconds() / 3600
    df['duration3'] = (df['punchout3'] - df['punchin3']).dt.total_seconds() / 3600
    
    # Replace negative or NaN values with 0
    duration_cols = ['duration1', 'duration2', 'duration3']
    for col in duration_cols:
        df[col] = df[col].fillna(0)
        df[col] = np.where(df[col] < 0, 0, df[col])
    
    # Calculate total daily hours
    df['daily_hours'] = df['duration1'] + df['duration2'] + df['duration3']
    
    return df

# Create employee-year features
def create_employee_year_features(df):
    # Group by employee_id and year
    grouped = df.groupby(['employee_id', 'year', 'department', 'Location', 'Supervisor'])
    
    # Create features
    employee_year_features = grouped.agg(
        days_worked=('date', 'count'),
        avg_daily_hours=('daily_hours', 'mean'),
        total_hours=('daily_hours', 'sum'),
        max_daily_hours=('daily_hours', 'max'),
        min_daily_hours=('daily_hours', lambda x: x[x > 0].min() if len(x[x > 0]) > 0 else 0),
        std_daily_hours=('daily_hours', 'std'),
        missing_punches=('daily_hours', lambda x: sum(x == 0)),
        overtime_days=('daily_hours', lambda x: sum(x > 8))
    ).reset_index()
    
    # Fill NaN values with 0 for std_daily_hours
    employee_year_features['std_daily_hours'] = employee_year_features['std_daily_hours'].fillna(0)
    
    return employee_year_features

# Calculate tenure features
def add_tenure_features(df):
    # Get the earliest year for each employee
    employee_first_year = df.groupby('employee_id')['year'].min().reset_index()
    employee_first_year.rename(columns={'year': 'first_year'}, inplace=True)
    
    # Merge to get first year for each employee
    df = pd.merge(df, employee_first_year, on='employee_id')
    
    # Calculate tenure
    df['tenure'] = df['year'] - df['first_year'] + 1
    
    return df

# Add pay rate change features
def add_pay_rate_changes(df):
    # Create a dataframe with previous year's pay rate
    prev_pay = df[['employee_id', 'year', 'pay_rate']].copy()
    prev_pay['year'] = prev_pay['year'] + 1
    prev_pay.rename(columns={'pay_rate': 'previous_pay_rate'}, inplace=True)
    
    # Merge with current data
    df = pd.merge(df, prev_pay, on=['employee_id', 'year'], how='left')
    
    # Calculate pay rate change
    df['previous_pay_rate'] = df['previous_pay_rate'].fillna(df['pay_rate'])  # For first year
    df['pay_rate_change'] = df['pay_rate'] - df['previous_pay_rate']
    
    # Avoid division by zero or null errors
    df['pay_rate_change_pct'] = np.where(
        df['previous_pay_rate'] > 0,
        (df['pay_rate_change'] / df['previous_pay_rate']) * 100,
        0
    )
    
    # Create a flag for missing payroll data
    df['has_payroll_data'] = ~pd.isnull(df['pay_rate']) | (df['pay_rate'] > 0)
    
    return df

# Main function to execute the pipeline
def prepare_data_for_ml():
    # Load data
    timekeeping_df, payroll_df = load_and_prepare_data()
    
    # Process time punches
    timekeeping_df = process_time_punches(timekeeping_df)
    
    # Create employee-year features
    employee_year_features = create_employee_year_features(timekeeping_df)
    
    # Merge with payroll data - using LEFT join to keep all employee-year records
    merged_df = pd.merge(employee_year_features, payroll_df, 
                         on=['employee_id', 'year'], how='left')
    
    # Handle missing pay rates
    merged_df['pay_rate'] = merged_df['pay_rate'].fillna(0)  # Fill missing pay rates with 0 or another strategy
    
    # Add tenure features
    merged_df = add_tenure_features(merged_df)
    
    # Add pay rate change features
    merged_df = add_pay_rate_changes(merged_df)
    
    # Optional: Create categorical features
    merged_df['department_encoded'] = merged_df['department'].astype('category').cat.codes
    merged_df['location_encoded'] = merged_df['Location'].astype('category').cat.codes
    merged_df['supervisor_encoded'] = merged_df['Supervisor'].astype('category').cat.codes
    
    return merged_df

# Execute the pipeline
if __name__ == "__main__":
    prepared_data = prepare_data_for_ml()
    
    # Display summary of prepared data
    print(f"Prepared data shape: {prepared_data.shape}")
    print("\nSample of prepared data:")
    print(prepared_data.head())
    
    # Show feature list
    print("\nFeatures available for ML:")
    for column in prepared_data.columns:
        print(f"- {column}")
    
    # Save prepared data
    prepared_data.to_csv('employee_data_prepared_for_ml.csv', index=False)
    print("\nPrepared data saved to 'employee_data_prepared_for_ml.csv'")

/var/folders/0b/kbxw3yp15078rb4blsyr1khw0000gn/T/ipykernel_75228/3110156780.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  timekeeping_df['date'] = pd.to_datetime(timekeeping_df['date'])
/var/folders/0b/kbxw3yp15078rb4blsyr1khw0000gn/T/ipykernel_75228/3110156780.py:31: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')
/var/folders/0b/kbxw3yp15078rb4blsyr1khw0000gn/T/ipykernel_75228/3110156780.py:31: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')
/var/folders/0b/kbxw3yp1507

Prepared data shape: (138, 23)

Sample of prepared data:
   employee_id  year                  department Location Supervisor  \
0           40  2024              ADMINISTRATION       SD     Joseph   
1          214  2024               IT DEPARTMENT       SD       Mary   
2          238  2021  LOGISTICS AND DISTRIBUTION       SD      Linda   
3          238  2022  LOGISTICS AND DISTRIBUTION       SD      Linda   
4          238  2023  LOGISTICS AND DISTRIBUTION       SD      Linda   

   days_worked  avg_daily_hours  total_hours  max_daily_hours  \
0           37         7.956306   294.383333         8.950000   
1            4         6.579167    26.316667         8.116667   
2           11         7.657576    84.233333         8.083333   
3           45         7.755556   349.000000         8.733333   
4           53         7.630503   404.416667         8.033333   

   min_daily_hours  ...  pay_rate  first_year  tenure  previous_pay_rate  \
0         7.716667  ...      16.0        20